# Pinecone - Primeros pasos

## Introducción <a name="intro"></a>

Lo primero que debemos realizar es crear una **API-Key** sobre la que poder auntenticarnos y, crear un proyecto sobre el que subir nuestros índices.

Por lo tanto, definimos la información necesaria:
+ **PINECONE_API_KEY**
+ **INDEX_NAME**

IMPORTANTE: Para poder trabajar con Pinecone desde Python debemos instalar el paquete que nos da acceso al cliente en la nube https://docs.pinecone.io/docs/python-client

In [ ]:
#pip install pinecone-client
#pip install python-dotenv 

In [32]:
import os
from pinecone import Pinecone
from dotenv import load_dotenv, find_dotenv
import numpy as np

In [26]:
load_dotenv(find_dotenv()) # busca un .env dentro de los archivos del repo (devuelve True si lo encuentra)

# load_dotenv(path) # si lo tenemos en otra localizacion (o con otro nombre) se puede pasar una ruta

True

In [23]:
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]

Una vez definidas nuestras variables constantes para poder interactuar con Pinecone, debemos lanzar el cliente, esto, se consigue a través de la función `Pinecone()` esta función recibe principalmente dos parámetros:
* `api_key`

De esta forma, ya hemos conectado el notebook al servidor de Pinecone

In [13]:
# Importamos Pinecode

pinecone = Pinecone(api_key=PINECONE_API_KEY)

Posteriormente, una vez conectados al servidor, podemos acceder a nuestros índices (creados previamente desde la interfaz de Pinecone) mediante la función `pinecone.list_indexes()` 

In [14]:
pinecone.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1024,
              'host': 'intropinecone-ska078u.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'intropinecone',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

También es posible crear un índice desde cero, todo desde el cliente en Python 

```python
from pinecone import Pinecone

pinecone = Pinecone(api_key=PINECONE_API_KEY)

index_name = "mi_index_prueba"

pinecone.create_index(
        index_name,
        dimension=1536,  
        metric='cosine'
    )

```

Además de poder acceder a su nombre, para controlar el estado del mismo es posible acceder a información como
* Status del índice
* Nombre del índice
* Tamaño del índice

Esto se consigue desde la función `pinecone.describe_index()`

In [16]:
pinecone.describe_index("intropinecone") # Devuelve un índice de valores

{'deletion_protection': 'disabled',
 'dimension': 1024,
 'host': 'intropinecone-ska078u.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'intropinecone',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

Como el índice puede ser algo privado, podemos definirlo en el fichero .env como una variable de entorno

In [27]:
INDEX_NAME = os.environ["INDEX"]

In [28]:
pinecone.describe_index(INDEX_NAME)

{'deletion_protection': 'disabled',
 'dimension': 1024,
 'host': 'intropinecone-ska078u.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'intropinecone',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [29]:
pinecone.describe_index(INDEX_NAME).dimension

1024

In [30]:
print(type(pinecone.describe_index(INDEX_NAME)))

<class 'pinecone.models.index_model.IndexModel'>


## Trabajando con Índices <a name="index"></a>

Antes de crear vectores propios con archivos PDF (el objetivo central de nuestro asistente conversacional), vamos a crear algunos vectores para subirlos a nuestro índice.

Ahora mismo nuestro índice tiene una dimensionalidad de 1024, vamos a crear algunos vectores aleatorios con Numpy para subir a nuestro índice de Pinecone (serían posibles números de Word Embeddings)

In [ ]:
vec_1 = np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist() # para subirlos es necesario pasarlos a lista
# vec_1 = np.random.normal(size = 1024).tolist()
vec_2 = np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()

In [34]:
vec_1[:3]

[-0.7647454842405442, -1.3488354302766006, -2.009611564951777]

Para poder subir estos dos vectores a Pinecone, debemos primero conectarnos al índice, esto se hace desde la función `pinecone.Index()`

In [35]:
index = pinecone.Index(INDEX_NAME)

Una vez que estemos conectados a un índice ya podremos realizar operaciones como:
* __Upsert__ - Insertar vectores
* __Query__  - Búsqueda por similaridad
* __Fetch__  - Recuperación de vectores
* __Delete__ - Borrar del índice
* __Update__ - Actualizar el/los elementos del vector

https://docs.pinecone.io/reference/query

Para subir vectores a un índice emplearemos la función `index.upsert()`. Esta función recibirá los siguientes parámetros principales:
* vectors: Lista de vectores

Esta lista de vectores, comúmente vendrá acompañada de campos que suelen introducirse como diccionario de datos (uno por cada vector) en donde se definen los siguientes campos clave:
+ id: nombre del índice
+ values : vector
+ metadata : Valores de metadatos de cada vector (útil para realizar búsqueda semántica)

In [36]:
# Posteriormente haremos esto de forma automatizada con la funcion from_documents

upsert_vectors = index.upsert(  # Insertamos vectores
    vectors = [
        
        {"id"       :"vec_1", 
         "values"   : vec_1,
         "metadata" : {"contenido" : "aprendizaje supervisado"}},
        
        {"id"       :"vec_2", 
         "values"   : vec_2,
         "metadata" : {"contenido" : "aprendizaje NO supervisado"}}        
    ]
)

### Ejercicio 1

Vamos a subir 100 vectores, a la colección, al igual que en el ejemplo que acabamos de ver, para el campo metadata, vamos a tomar los posibles valores en el campo "contenido".
+ Las herramientas del científico de datos
+ Machine Learning
+ Estadística para el Data Science

El valor de "contenido" será aleatorio

#### Solución

## Funciones de búsqueda en vectores <a name="query"></a>

Por el momento, nuestras _querys_ serán sin lenguaje natural, ya que, aún no hemos subido ningún archivo de texto, nuestros vectores no son ni más ni menos que números que siguen una distribución normal, pero, este es el objetivo, cuando llegue una preguna en lenguaje natural, se convertirá en Embeddings y, se buscarán vectores por similaridad.

Para realizar una búsqueda por similaridad en nuestro índice sobre los vectores, podemos hacer uso de la función `index.query()` la cuál recibe otro vector como parámetro y, el parámetro `top_k` que es el número de resultados a desplegar.

In [37]:
query_vector = np.random.normal(size = \
                                int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()

In [38]:
query_vector[:20]

[-1.15238549437899,
 -0.40361093913706975,
 0.5034572242501693,
 -1.7190467936623295,
 -0.06837911192864113,
 0.7168079918890528,
 -1.1348130225776514,
 0.952719161052057,
 1.2281627829356319,
 -0.9251060491499601,
 -0.2496803461757182,
 -0.05049975044256642,
 -1.0062360728560742,
 -1.220606780062563,
 -0.7573371683350697,
 -1.657236960976515,
 -1.9726670495976724,
 0.9786944207943854,
 -0.15507729268785386,
 -0.8225234755020444]

In [39]:
ask_query = index.query(
    vector = query_vector, # Vector de búsqueda,
    top_k  = 1 # Solo el resultado con la similaridad más alta
)

El resultado almacenado en esta variable será simplemente los id de los vectores con la similariadad más alta, tantos como le hayamos pasado al parámetro `top_k`

In [40]:
ask_query

{'matches': [{'id': 'vec_2', 'score': 0.016998712, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

In [41]:
index.query(
    vector = query_vector, # Vector de búsqueda,
    top_k  = 2 # Solo el resultado con la similaridad más alta
)

{'matches': [{'id': 'vec_2', 'score': 0.016998712, 'values': []},
             {'id': 'vec_1', 'score': -0.0183637626, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

También es posible hacer una busqueda semántica mediante metadatos. https://docs.pinecone.io/docs/metadata-filtering.

La búsqueda semántica te va a permitir filtrar para que solo busque en vectores que cumplan ciertas condiciones.

In [42]:
# Incluimos mas vectores de prueba

index.upsert([
    ("A", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
     {"genre": "comedy", "year": 2020}),
    
    ("B", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(),
     {"genre": "documentary", "year": 2019}),
    
    ("C", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(),
     {"genre": "comedy", "year": 2019}),
    
    ("D", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
     {"genre": "drama"}),
    
    ("E", np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
     {"genre": "drama"})
])

{'upserted_count': 5}

In [43]:
query_vector = np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()

In [44]:
index.query(
    vector=query_vector,
    filter={
        "genre": {"$eq": "comedy"},
        "year": 2019
    },
    top_k=1,
    include_metadata=True # para que filtre por metadatos hay que añadir este parametro
)

{'matches': [{'id': 'C',
              'metadata': {'genre': 'comedy', 'year': 2019.0},
              'score': 0.00851308741,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}}

Para borrar vectores, se puede emplear la función `index.delete()` Es posible eliminar directamente IDs concretos sobre el campo `ids` o borrar por metadatos sobre el campo `filter`

In [45]:
index.delete(ids=["A", "B"])

{}

También es posible borrar vectores mediante el filtrado de metadatos, no obstante esta operación no está habilitada para las cuentas que tengan un entorno de computación gcp-starter: https://docs.pinecone.io/docs/metadata-filtering#deleting-vectors-by-metadata-filter

```python
index.delete(
    filter={
        "genre": {"$eq": "documentary"},
        "year": 2019
    }
)
```

Para borrar todos los vectores de un índice podemos hacer uso del parámetro `delete_all`, no obstante, tampoco es posible realizar esta operación en cuentas tipo Starter.

```python
# borrar todos los vectores
index.delete(delete_all=True)
```

Para actualizar un vector se puede realizar desde la función `index.update()` https://docs.pinecone.io/docs/manage-data#updating-records

In [46]:
# Creamos un vector para comprobar la operación update

vec_test = np.random.normal(size = int(pinecone.describe_index(INDEX_NAME).dimension)).tolist()

upsert_vectors = index.upsert(  # Insertamos vectores
    vectors = [
        
        {"id"       :"test", 
         "values"   : vec_test,
         "metadata" : {"contenido" : "aprendizaje supervisado"}},    
    ]
)

In [47]:
# Actualizamos el vector
index.update(id="test", 
             values= np.ones(int(pinecone.describe_index(INDEX_NAME).dimension)).tolist(), 
             set_metadata={"contenido": ["aprendizaje supervisado", "Estadística"],
                           "fecha" : "11/10/2024"}
                           )

{}